In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.image import resize


In [ ]:
model = tf.keras.models.load_model("./Melody_Mosaic/Trained_model.keras") ## Load model according to your path

In [ ]:
file_path="./classical.00000.wav" # set this path to music file for testing according to your folder structure
classes =['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']
from IPython.display import Audio
y,sr=librosa.load(file_path,sr=44100)
Audio(data=y,rate=sr)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def load_and_preprocess_file(file_path,target_shape=(140,140)):
  data=[]
  audio_data, sample_rate = librosa.load(file_path,sr=None)
  ##preprocessing start
  chunk_duration = 4
  overlap_duration = 2
  # convert durattion to sample
  chunk_samples = chunk_duration * sample_rate
  overlap_samples = overlap_duration * sample_rate

  #calculate the number of chunks
  num_chunks = int(np.ceil((len(audio_data)-chunk_samples)/(chunk_samples-overlap_samples)))+1
   #iterate over each chunks
  for i in range(num_chunks):
                    #calculate start and end index of the chunk
                    start = i*(chunk_samples-overlap_samples)
                    end = min(start + chunk_samples, len(y))
                    chunk = audio_data[start:end]
                    mel_spectrogram = librosa.feature.melspectrogram(y=chunk, sr=sample_rate)
                    mel_spectrogram = resize(np.expand_dims(mel_spectrogram,axis=-1),target_shape)
                    data.append(mel_spectrogram)
  return np.array(data)

In [ ]:
X_Test = load_and_preprocess_file(file_path)


In [ ]:
X_Test.shape

(15, 140, 140, 1)

In [ ]:
def model_prediction(X_test):
    y_pred = model.predict(X_test)
    predicted_categories = np.argmax(y_pred, axis=1)

    # Find the most frequent predicted class
    unique_elements, counts = np.unique(predicted_categories, return_counts=True)
    max_count = np.max(counts)
    max_elements = unique_elements[counts == max_count]

    return max_elements[0]


In [ ]:
c_index=model_prediction(X_Test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [ ]:
c_index

1

In [ ]:
print("Model output--",classes[c_index])

Model output-- classical
